In [1]:
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk
! pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5531aaaf08b7c2a46f425328c1c17f09b7a24ea38d10e74c254375039435db1b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532

In [2]:
import wandb
from transformers import DataCollatorForSeq2Seq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm


device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=2
T5_model = "t5-base"
sep='<s>'

2024-04-25 09:31:48.519546: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 09:31:48.519645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 09:31:48.666941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
def fun(word1, word2, dp):

    # check if the result is already present in the dp table
    if dp[len(word2)][len(word1)] != -1:
        return dp[len(word2)][len(word1)]          

    # if both words are empty, then the number of operations required is 0
    if len(word1) == 0 and len(word2) == 0:
        return 0

    # if one of the words is empty, then the number of operations required is the length of the other word
    if len(word2) == 0:
        return len(word1)

    if len(word1) == 0:
        return len(word2)

    # add: insert a character in word1 to match word2
    add = fun(word1, word2[1:], dp) + 1

    # delete: delete a character in word1 to match word2
    delete = fun(word1[1:], word2, dp) + 1

    # replace: replace a character in word1 to match word2
    if word1[0] == word2[0]:
        # if the firsr character is same, then the number of operations will not increase
        replace = fun(word1[1:], word2[1:], dp)
    else:
        # else, the number of operations will increase
        replace = fun(word1[1:], word2[1:], dp) + 1

    # take the minimum of the three recursive calls and store in the dp table
    dp[len(word2)][len(word1)] = min(min(add, delete), replace)

    # return the result
    return dp[len(word2)][len(word1)]


def minDistance(word1: str, word2: str) -> int:
    # create the dp table with -1 as initial values
    dp = [[-1 for i in range(len(word1)+1)] for j in range(len(word2)+1)]

    # call the recursive function and return the result
    return fun(word1, word2, dp)


In [4]:
data=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True,encoding='utf-8')


In [5]:
def get_split_data(data,if_train=True):
    global label_encoder

    # y = data["spoiler"]
    Y=[]
    # for i in y:
    #     Y.append(" ".join(i))
    y=0
    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords','spoiler','tags']]
    dic1=new_data.to_dict(orient='records')

    dic=[]
    for i in dic1:
        # print(i)
        if('phrase' not in i['tags']):
            del i['postText']
            del i['targetParagraphs']
            del i['targetTitle']
            del i['targetDescription']
            del i['targetKeywords']
            del i['tags']
            del i['spoiler']
            del i
            continue

        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        j={}
        j['question']="question : "+i['postText']+"?"
        j['context']="context : "+i['targetParagraphs']
        dic.append(j)
        b=" ".join(i['spoiler'])
        if(b==" "):
            y+=1
        Y.append(" ".join(i['spoiler']))

        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']
        del i['tags']
        del i['spoiler']
        # break
    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
    # print(dic)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)
    print(y)
    # return X_q[:100],X_c[:100],Y[:100]
    return X_q,X_c,Y

x_test_q,x_test_c,y_test = get_split_data(data,False)
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


0


In [6]:
M1="/kaggle/input/t5-dataset-proj/T5/checkpoint-3200"
M2="/kaggle/input/t5-passage/T5_overall"
M3="/kaggle/input/t5-multi/T5_overall"
M4="/kaggle/input/t5-phrase/T5_overall"
M5="/kaggle/input/t5-proj-1/T5_overall"
M6="/kaggle/input/t5-qa/T5_overall"

tokenizer1 = AutoTokenizer.from_pretrained(M1)
model1 = AutoModelWithLMHead.from_pretrained(M1)

tokenizer2 = AutoTokenizer.from_pretrained(M2)
model2 = AutoModelWithLMHead.from_pretrained(M2)

tokenizer3 = AutoTokenizer.from_pretrained(M3)
model3 = AutoModelWithLMHead.from_pretrained(M3)

tokenizer4 = AutoTokenizer.from_pretrained(M4)
model4 = AutoModelWithLMHead.from_pretrained(M4)

tokenizer5 = AutoTokenizer.from_pretrained(M5)
model5 = AutoModelWithLMHead.from_pretrained(M5)

tokenizer6 = AutoTokenizer.from_pretrained(M6)
model6 = AutoModelWithLMHead.from_pretrained(M6)

tokenizer5 = AutoTokenizer.from_pretrained(M5)
model_toke={0:[model1,tokenizer1],1:[model2,tokenizer2],2:[model3,tokenizer3],3:[model4,tokenizer4],4:[model5,tokenizer5],5:[model6,tokenizer6]}
models={0:M1,1:M2,2:M3,3:M4,4:M5,5:M6}

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [7]:
model1.to(device)
model2.to(device)
model3.to(device)
model4.to(device)
model5.to(device)
model6.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [8]:
from tqdm import tqdm

In [9]:
decoded=[]
for i in range(len(model_toke)):
    model=model_toke[i][0].to(device)
    tokenizer=model_toke[i][1]
    cur_decoded=[]
    for i in tqdm(range(len(test_df))):

    # for index, row in tqdm(test_df.iterrows()):
        row=test_df.iloc[i]
        index=i
        # input_text = row['question'] + sep + row['context']
        input_ids = tokenizer.encode(row['question'],row['context'], return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
        cur_decoded.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        # if(index==10):
        #     break
    decoded.append(cur_decoded)

100%|██████████| 423/423 [01:55<00:00,  3.67it/s]


In [10]:
spoiler=test_df['spoiler'].tolist()

In [11]:
# decoded=[]
# spoiler=[]
# for i in tqdm(range(len(test_df))):
# # for i in range(len(test_df)):
#     # print(test_df.iloc[i])
        
#     q=test_df.iloc[i]['question']
#     c=test_df.iloc[i]['context']
#     enc1=tokenizer1(q,c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out1=model1.generate(input_ids=enc1['input_ids'].to(device),attention_mask=enc1['attention_mask'].to(device),max_length=512,num_beams=4,early_stopping=True)
#     dec1=tokenizer1.decode(out1[0],skip_special_tokens=True)

#     enc2=tokenizer2(q,c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out2=model2.generate(input_ids=enc2['input_ids'].to(device),attention_mask=enc2['attention_mask'].to(device),max_length=512,num_beams=4,early_stopping=True)
#     dec2=tokenizer2.decode(out2[0],skip_special_tokens=True)

#     enc3=tokenizer3(q,c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out3=model3.generate(input_ids=enc3['input_ids'].to(device),attention_mask=enc3['attention_mask'].to(device),max_length=512,num_beams=4,early_stopping=True)
#     dec3=tokenizer3.decode(out3[0],skip_special_tokens=True)
#     penalty=200
#     edit1=0
#     if(len(dec1)<3):
#         edit1=penalty
#     decs=[dec1,dec2,dec3]
#     for j in range(1,3):
#         edit1+=minDistance(decs[0],decs[j])
#     edit2=0
#     if(len(dec2)<3):
#         edit2=penalty
#     for j in range(2):
#         edit2+=minDistance(decs[1],decs[j])
#     edit3=0
#     if(len(dec3)<3):
#         edit3=penalty
#     for j in range(2):
#         edit3+=minDistance(decs[2],decs[j])
    
#     if(edit1<=edit2 and edit1<=edit3):
#         dec=dec1
#     elif(edit2<=edit1 and edit2<=edit3):
#         dec=dec2
#     else:
#         dec=dec3
#     print(edit1,edit2,edit3)
#     print(decs)
#     decoded.append(dec)
#     spoiler.append(test_df.iloc[i]['spoiler'])
#     # break
#     if(i==10):
#         break
# decoded,spoiler

In [12]:
ensemble=[]
for i in range(len(spoiler)):
    predicted=[decoded[j][i] for j in range(len(decoded))]
    edit=[]
    for j in range(len(decoded)):
        e=0
        for k in range(len(decoded)):
            e+=minDistance(decoded[j],predicted[k])
        edit.append(e)
    
    arg_min=np.argmin(edit)
    ensemble.append(decoded[arg_min][i])



In [13]:
decoded.append(ensemble)
models[len(model_toke)]="Ensemble"

In [14]:
# blue
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])
    # print(len(preds),len(refs))
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=refs)
    print(calculate_bleu(results['brevity_penalty'],results['precisions']))
    print(results)  


/kaggle/input/t5-dataset-proj/T5/checkpoint-3200


[0.38469886055344554, 0.3258812033364205, 0.26275636587854634, 0.2037686085123217]
{'bleu': 0.20376860851232173, 'precisions': [0.3846988605534455, 0.27605633802816903, 0.16404077849860982, 0.09896432681242807], 'brevity_penalty': 1.0, 'length_ratio': 1.56984667802385, 'translation_length': 1843, 'reference_length': 1174}
/kaggle/input/t5-passage/T5_overall
[0.10902394106813999, 0.08451018739528958, 0.06517848062090248, 0.047231882558415005]
{'bleu': 0.047231882558415005, 'precisions': [0.10902394106813997, 0.06550828839624526, 0.03569096844396083, 0.019523809523809523], 'brevity_penalty': 1.0, 'length_ratio': 4.625212947189097, 'translation_length': 5430, 'reference_length': 1174}
/kaggle/input/t5-multi/T5_overall
[0.16341991341991344, 0.12953828712529797, 0.09974740886363515, 0.0743472346284192]
{'bleu': 0.07434723462841918, 'precisions': [0.1634199134199134, 0.10268129189518586, 0.055153395380903136, 0.03301343570057581], 'brevity_penalty': 1.0, 'length_ratio': 3.148211243611584, 't

In [15]:
! pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
# rouge
for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])

    # print(len(preds),len(refs))
    rouge = evaluate.load("rouge")
    results = rouge.compute(predictions=preds, references=refs)
    print(results)

/kaggle/input/t5-dataset-proj/T5/checkpoint-3200


{'rouge1': 0.6416259028095049, 'rouge2': 0.4443269230596591, 'rougeL': 0.6417239515081832, 'rougeLsum': 0.6415613933340374}
/kaggle/input/t5-passage/T5_overall
{'rouge1': 0.26109182061690317, 'rouge2': 0.16790017174669436, 'rougeL': 0.26212402271436974, 'rougeLsum': 0.26194853838500864}
/kaggle/input/t5-multi/T5_overall
{'rouge1': 0.40705104585886176, 'rouge2': 0.28431529621778345, 'rougeL': 0.4074256821769692, 'rougeLsum': 0.40753749550377744}
/kaggle/input/t5-phrase/T5_overall
{'rouge1': 0.6644200500396844, 'rouge2': 0.45322817141757477, 'rougeL': 0.6635297195580883, 'rougeLsum': 0.6651020343349573}
/kaggle/input/t5-proj-1/T5_overall
{'rouge1': 0.6563224372574881, 'rouge2': 0.4575794143993428, 'rougeL': 0.6573965173255956, 'rougeLsum': 0.6563816549258882}
/kaggle/input/t5-qa/T5_overall
{'rouge1': 0.5214996175295628, 'rouge2': 0.3562580950951847, 'rougeL': 0.5221333041485113, 'rougeLsum': 0.521507311756666}
Ensemble
{'rouge1': 0.6416259028095049, 'rouge2': 0.4443269230596591, 'rougeL'

In [17]:
# bertscore
bert = evaluate.load("bertscore")

for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])

    results = bert.compute(predictions=preds, references=refs,lang='en')
    precision = np.array(results['precision'])
    recall = np.array(results['recall'])
    f1 = np.array(results['f1'])

    print("Precision: ",precision.mean())
    print("Recall: ",recall.mean())
    print("F1: ",f1.mean()) 

/kaggle/input/t5-dataset-proj/T5/checkpoint-3200


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision:  0.9243141102734469
Recall:  0.9296755203118561
F1:  0.9265343869549727
/kaggle/input/t5-passage/T5_overall
Precision:  0.8418840324906874
Recall:  0.8701748154687543
F1:  0.8551418565689249
/kaggle/input/t5-multi/T5_overall


Precision:  0.8510144530458653
Recall:  0.8823224014424264
F1:  0.8656780402711097
/kaggle/input/t5-phrase/T5_overall
Precision:  0.9321692013853267
Recall:  0.9278949549857606
F1:  0.9296170013734353
/kaggle/input/t5-proj-1/T5_overall
Precision:  0.9259436853388523
Recall:  0.9314364108915306
F1:  0.9282238941384263
/kaggle/input/t5-qa/T5_overall
Precision:  0.8993776270120138
Recall:  0.9102405854996215
F1:  0.9042587872092606
Ensemble
Precision:  0.9243141102734469
Recall:  0.9296755203118561
F1:  0.9265343869549727


In [18]:
# meteor

meteor = evaluate.load("meteor")
for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])

    results = meteor.compute(predictions=preds, references=refs)
    print(results)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


/kaggle/input/t5-dataset-proj/T5/checkpoint-3200
{'meteor': 0.5519949004729426}
/kaggle/input/t5-passage/T5_overall
{'meteor': 0.3352987864585139}
/kaggle/input/t5-multi/T5_overall
{'meteor': 0.41290908159211714}
/kaggle/input/t5-phrase/T5_overall
{'meteor': 0.544082077552373}
/kaggle/input/t5-proj-1/T5_overall
{'meteor': 0.5627012516381681}
/kaggle/input/t5-qa/T5_overall
{'meteor': 0.45974542846255606}
Ensemble
{'meteor': 0.5519949004729426}
